In [1]:
import Bio
from Bio import Phylo
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
#import pylab
import numpy as np
import networkx as nx
#import matplotlib.pyplot as plt
import pydot
import pygraphviz as pgv
from networkx.drawing.nx_pydot import graphviz_layout
from networkx import all_pairs_shortest_path_length
import hyperlib
from hyperlib.embedding.treerep import treerep
from hyperlib.embedding.sarkar import sarkar_embedding
from hyperlib.utils.multiprecision import poincare_dist
from hyperlib.utils.multiprecision import poincare_reflect0
#from hyperlib.manifold.lorentz import Lorentz
#from hyperlib.manifold.poincare import Poincare
import mpmath as mpm
from scipy.special import expit
import tensorflow as tf
from tensorflow import keras

In [2]:
file_name = './205_na_aln.fa'
file_format = 'fasta'

In [3]:
aln = AlignIO.read(file_name, file_format)
#print(aln)

In [4]:
labels = {i:seq.id for i, seq in enumerate(aln)}
#labels

In [5]:
aln_length = aln.get_alignment_length()

sotto il modulo treeconstruction di Phylo c'è questo DistanceCalculator che calcola automaticamente le distanze secondo un modello dato come input, 'identity' di default.
il metodo get_distance produce un oggetto di tipo DistanceMatrix, che va quindi converito con numpy.

In [6]:
calc = DistanceCalculator('trans')
dm = calc.get_distance(aln)
#print(dm)
type(dm)

Bio.Phylo.TreeConstruction.DistanceMatrix

In [7]:
dist_mat = np.array(dm)
#dist_mat

In [8]:
constructor = DistanceTreeConstructor()
ptree = constructor.upgma(dm)
ptree.ladderize()
#Phylo.draw(ptree)

In [9]:
#Phylo.draw(ptree)

In [10]:
htree = treerep(dist_mat,return_networkx = True, tol = 0.0000000000000000000000001)
htree


In [11]:
htree.number_of_nodes()

224

In [12]:
pesi_aggiornati = [(edge[0], edge[1], {'weight' : expit(htree.get_edge_data(edge[0], edge[1])['weight'])}) for edge in list(htree.edges)]
#pesi_aggiornati

In [13]:
for i in pesi_aggiornati:
    htree.update([i])
htree.get_edge_data(4, 49)

In [14]:
htree[3]

AtlasView({33: {'weight': 0.5}})

In [15]:
htgraph = nx.nx_agraph.to_agraph(htree)

In [16]:
#htgraph.layout?

In [17]:
htgtwopi = htgraph
htgtwopi.layout(prog = 'twopi') #dot, neato
#htgtwopi

In [18]:
print(htree)

Graph with 224 nodes and 223 edges


Infine il codice suggerito nell'esempio della libreria su github che dovrebbe fare l'embedding iperbolico nella sfera di Poincaré

In [19]:
root = 0 # label of root node
tau = 0.2 # scaling factor for edges
embed_2D = sarkar_embedding(htree, root)

# calculate hyperbolic distances from the embedding
poincare_dist(embed_2D[0,:], embed_2D[1,:])

mpf('182.3930140184730296830577818620853629355128624706745952553613730108328677888921117463192282011772899928295324397826630587037104171664147009266325151865162568555755224573700242316251616213378906730657917555326376907130667781711161946561635505501557502749903012449283476284226224131427363125102892699180013462569534431948250584')

In [20]:
#embed_2D

In [21]:
embed_2D[2, :] - embed_2D[1, :]

matrix(
[['-0.000003423767373985497984751829356124747085063760477591075834961990692450826259296341224306019947944315079684713212979073356493723219441847436171787303741494917810710510838979120572783480491943591745672352197053777428079487765954959897686032287107621031035249424852628182889140779419344143473784514951479874114276130901639076456', '0.005231774456298183848868269141325044776261945825202421062376019137696801914050044753010894185013976064400681042387321046914197558385284662849526699172849311596174751965364629275793968615729134769901644605393966053989845375783867845587629778476655030500633610682487087931944236830809385226817972148091128030929594756482837622']])

# **Codice di preparazione per H-depp**

In [22]:
import sys 
from pathlib import Path

module_path = str(Path.cwd().parents[0])
sys.path.append(module_path)

In [23]:
from src.data_generator import DataGenerator
from src.loss_function import hdepp_loss_couple
from src.loss_function import hdepp_loss

In [24]:
import tensorflow_riemopt

In [25]:
original_nodes = len(aln)

In [26]:
tree_dist = dict(all_pairs_shortest_path_length(htree))

In [27]:
reference_tensor = tf.convert_to_tensor([[tree_dist[i][j] for j in range(0, original_nodes)] for i in range(0, original_nodes)])

In [28]:
reference_tensor.get_shape()

TensorShape([205, 205])

In [29]:
reference_tensor[0][3]

<tf.Tensor: shape=(), dtype=int32, numpy=2>

**Distortions of the old embeddings**

In [30]:
emb_dist = {}

#note that i use poincare_dist to calculate the embeddings distance
#we might want to calculate it differently
for i in range (0, embed_2D.rows):
    temp_dict = {}
    for j in range (0, embed_2D.rows):
        jnode = list(htree.nodes)[j]
        temp_dict[jnode] = poincare_dist(embed_2D[i,:], embed_2D[j,:])
    inode = list(htree.nodes)[i]
    emb_dist[inode] = temp_dict

emb_dist[0][3]

mpf('183.5706887125437275012669761316737293472617396839872234836643717924866364940717333990487804252432743470169131907996470112513843975258455353536347153842456837294554564308593777892772591629778012512753187676099243081539128496201698145316497961309813613126520098908361853240154806379715122640915085424493210542363988208267386476')

In [31]:
def distortion_func(e_dist, t_dist):
    if (t_dist != 0):
        return np.square(( e_dist / t_dist ) - 1) 
    else: return 0

In [32]:
distortion = {}

for i in range (0, embed_2D.rows):
    temp_dict = {}
    for j in range (0, embed_2D.rows):
        jnode = list(htree.nodes)[j]
        temp_dict[jnode] = distortion_func(emb_dist[i][j], tree_dist[i][j])
    inode = list(htree.nodes)[i]
    distortion[inode] = temp_dict
    
distortion[0][3]

mpf('4.95021677894868878817336334564908546740134272664065431651070898600347645811634249429357125024968430692939913138850955383342030083736693737043879145601905638699703476019557666213811223948491572384014138730421419884895025299168795358490885744883002682359412122892913711795255528735768967170928689282513655583160863458363731869')

**Encoding the sequences**

In [33]:
aln[0].seq

Seq('ATGAGCCAAGAAGAAAAGTTACCAAAGATTCTGATCGTTGAAGACGACGAGCGT...TTG')

In [34]:
hot_dict = {'A' : np.array([0, 0, 0, 1]),
           'T' : np.array([0, 0, 1, 0]),
           'G' : np.array([0, 1, 0, 0]),
           'C' : np.array([1, 0, 0, 0]),
           '-' : np.array([0, 0, 0, 0]),}
hot_dict[aln[0].seq[2]]

array([0, 1, 0, 0])

In [35]:
encoded_seqs = [np.array([hot_dict[car] for car in alignment.seq]) for alignment in aln]

In [36]:
model_inputs = [[(encoded_seqs[i], encoded_seqs[j], tree_dist[i][j]) for j in tree_dist[i] if j < original_nodes] for i in tree_dist if i < original_nodes]

**Train and validation sets**

In [37]:
#lista_IDs_train = [[i, j] for j in range(0, 22) for i in range(0, 22)]
#lista_IDs_val = [[i, j] for j in range(22, 29) for i in range(22, 29)] #what should I use for validation?

In [38]:
lista_IDs_train = [i for i in range(0, original_nodes-30)]

In [39]:
lista_IDs_val = [i for i in range(original_nodes-30, original_nodes)]

In [40]:
params = {'dim': (aln_length, ),
          'batch_size': 4,
          'n_classes': aln_length, #shouldn't be categorical
          'n_channels': 4,
          'shuffle': True}
#da correggere

In [41]:
train_ds_gen = DataGenerator(lista_IDs_train, model_inputs, **params)
test_ds_gen = DataGenerator(lista_IDs_val, model_inputs, **params)

In [42]:
from hyperlib.nn.layers.lin_hyp import LinearHyperbolic
from hyperlib.nn.optimizers.rsgd import RSGD
from hyperlib.manifold.poincare import Poincare

In [43]:
from tensorflow import keras

In [44]:
#tensorflow_riemopt.optimizers.RiemannianSGD??

In [45]:
#optimizer = keras.optimizers.SGD(learning_rate=1e-9, clipnorm = 1.0, clipvalue = 0.5)
#optimizer = RSGD(learning_rate = 0.1)
optimizer = tensorflow_riemopt.optimizers.RiemannianSGD(learning_rate = 1e-18, clipnorm = 1.0, clipvalue = 0.5)
loss_fn = hdepp_loss

In [46]:
#tf.keras.layers.LSTM?

In [47]:

model = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(input_shape=(714, 4)),
    tf.keras.Input((714, 4), 4),
    
    tf.keras.layers.Conv1D(8, 1, kernel_regularizer = 'L2'),
    tf.keras.layers.Conv1D(3, 5, kernel_regularizer = 'L2'),
    tf.keras.layers.Conv1D(2, 5, kernel_regularizer = 'L2'),
    tf.keras.layers.LSTM(15)
    #LinearHyperbolic(10, Poincare(), 1)
    #tf.keras.layers.Dense(10)
])
'''
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    #sparse vs non sparse
)

model.fit(
    train_ds_gen,
    epochs=9,
    validation_data = test_ds_gen
)
'''

'\nmodel.compile(\n    optimizer=tf.keras.optimizers.Adam(0.001),\n    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),\n    metrics=[tf.keras.metrics.CategoricalAccuracy()],\n    #sparse vs non sparse\n)\n\nmodel.fit(\n    train_ds_gen,\n    epochs=9,\n    validation_data = test_ds_gen\n)\n'

In [48]:
#optimizer.apply_gradients?

In [49]:
#attempts at building the training loop
epochs = 3

for epoch in range(epochs):
    
    for step, (x_batch, y_batch) in enumerate(train_ds_gen):
        with tf.GradientTape() as tape:
            tape.watch(model.trainable_weights)
            #x_batch, y_batch = tf.convert_to_tensor(x_batch, dtype = tf.double), tf.convert_to_tensor(y_batch)
            logits = model(x_batch, training = True)
            loss_value = tf.convert_to_tensor(hdepp_loss(logits, y_batch, reference_tensor))
            #loss_value = tf.convert_to_tensor([[44, 44, 44], [3, 3, 3], [4, 5, 6]])

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(list(zip(grads, model.trainable_weights)))
        #for gw in zip(grads, model.trainable_weights):
        #    optimizer.apply_gradients(gw)
        print(step, loss_value)
    print('end of epoch number: ', epoch + 1)

#, unconnected_gradients=tf.UnconnectedGradients.ZERO

0 tf.Tensor(12.0, shape=(), dtype=float32)
1 tf.Tensor(nan, shape=(), dtype=float32)
2 tf.Tensor(nan, shape=(), dtype=float32)
3 tf.Tensor(nan, shape=(), dtype=float32)
4 tf.Tensor(nan, shape=(), dtype=float32)
5 tf.Tensor(nan, shape=(), dtype=float32)
6 tf.Tensor(nan, shape=(), dtype=float32)
7 tf.Tensor(nan, shape=(), dtype=float32)
8 tf.Tensor(nan, shape=(), dtype=float32)
9 tf.Tensor(nan, shape=(), dtype=float32)
10 tf.Tensor(nan, shape=(), dtype=float32)
11 tf.Tensor(nan, shape=(), dtype=float32)
12 tf.Tensor(nan, shape=(), dtype=float32)
13 tf.Tensor(nan, shape=(), dtype=float32)
14 tf.Tensor(nan, shape=(), dtype=float32)
15 tf.Tensor(nan, shape=(), dtype=float32)
16 tf.Tensor(nan, shape=(), dtype=float32)
17 tf.Tensor(nan, shape=(), dtype=float32)
18 tf.Tensor(nan, shape=(), dtype=float32)
19 tf.Tensor(nan, shape=(), dtype=float32)
20 tf.Tensor(nan, shape=(), dtype=float32)
21 tf.Tensor(nan, shape=(), dtype=float32)
22 tf.Tensor(nan, shape=(), dtype=float32)
23 tf.Tensor(nan, sh

In [50]:
model.trainable_weights

[<tf.Variable 'conv1d/kernel:0' shape=(1, 4, 8) dtype=float32, numpy=
 array([[[nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan]]], dtype=float32)>,
 <tf.Variable 'conv1d/bias:0' shape=(8,) dtype=float32, numpy=array([nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)>,
 <tf.Variable 'conv1d_1/kernel:0' shape=(5, 8, 3) dtype=float32, numpy=
 array([[[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],
 
        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]],
 
        [[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan],
        